# PPMI

In [1]:

import sys,os
# sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(__file__)))) # 添加上层目录到 sys.path
# from data_process import data_process,Labels
# from data_process import X_test,X_train
# from data_process import y_test,y_train,Labels
# from data_process import X_test_MCI,X_train_MCI,y_test_MCI,y_train_MCI
# # from dimension_reduction import X_train,X_test
from dimension_reduction_lda import dimension_reduction,visualization
import numpy as np
from sklearn import svm
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
dim=2

def _svm(X_train,y_train,X_valid,y_valid,X_test,y_test):
        clf=svm.SVC(kernel='linear',C=1.0,random_state=0,decision_function_shape='ovr')
        clf.fit(X_train,y_train)
        y_pred=clf.predict(X_test)
        train_acc=accuracy_score(y_train,clf.predict(X_train))
        valid_acc=accuracy_score(y_valid,clf.predict(X_valid))
        test_acc=accuracy_score(y_test,y_pred)
        # print('train:',accuracy_score(y_train,clf.predict(X_train)))
        # print('validation',accuracy_score(y_valid,clf.predict(X_valid)))
        # print('test:',accuracy_score(y_test,clf.predict(X_test)))
        # print(classification_report(y_test,y_pred,target_names=Labels))
        return train_acc,valid_acc,test_acc
seeds=[0,1,2,3,4]
train_accs,valid_accs,test_accs=[],[],[]
# if __name__=='__main__':
for seed in seeds:
    # X_train, y_train, X_valid, y_valid, X_test, y_test=[],[],[],[],[],[]
    X_train, y_train, X_valid, y_valid, X_test, y_test=data_process(seed)
    print('降维前的svm的表现:')
    _svm(X_train,y_train,X_valid, y_valid,X_test,y_test)
    print('降维后svm的表现:')

    X=np.concatenate((X_train,X_valid,X_test),axis=0)
    y=np.concatenate((y_train,y_valid,y_test),axis=0)
    X_reduced=dimension_reduction(X,y,dim=120,method='PCA')
    X_train_reduced=X_reduced[:X_train.shape[0],:]
    X_valid_reduced=X_reduced[X_train.shape[0]:X_train.shape[0]+X_valid.shape[0],:]
    X_test_reduced=X_reduced[X_train.shape[0]+X_valid.shape[0]:,:]

    # X_train_reduced=dimension_reduction(X_train_reduced,y_train,dim=30,method='Laplacian')
    # X_train_reduced,proj_mat=dimension_reduction(X_train_reduced,y_train,dim=1,method='LDA')
    # visualization(X_train_reduced,y_train,dim=dim,method='PCA',class_num=2)
    # X_test_reduced=dimension_reduction(X_test,y_test,dim=2,method='Isomap')
    # X_test_reduced=dimension_reduction(X_test_reduced,y_test,dim=30,method='Laplacian')
    # X_test_reduced=np.dot(X_test_reduced,proj_mat)
    # visualization(X_test_reduced,y_test,dim=dim,method='PCA',class_num=2)
    # _svm(X_train_reduced,y_train,X_test_reduced,y_test)
    train_acc,valid_acc,test_acc=_svm(X_train_reduced,y_train,X_valid_reduced,y_valid,X_test_reduced,y_test)
    train_accs.append(train_acc)
    valid_accs.append(valid_acc)
    test_accs.append(test_acc)
print('train_accs:',train_accs)
print('valid_accs:',valid_accs)
print('test_accs:',test_accs)
print('train_acc:',np.mean(train_accs),np.std(train_accs))
print('valid_acc:',np.mean(valid_accs),np.std(valid_accs))
print('test_acc:',np.mean(test_accs),np.std(test_accs))

Random seed is set to 0.
{'AD': 0, 'MCI': 1, 'MCIn': 2, 'MCIp': 3, 'NC': 4}


NameError: name 'data_process' is not defined

# ADNI

In [2]:
# 读入.mat文件, 划分测试集与训练集

import scipy.io
import numpy as np
import sys
from sklearn.preprocessing import StandardScaler
# if len(sys.argv)!=2:
#     print('Usage: python data_process.py <dataset_path>')
#     sys.exit(1)

# file_path=sys.argv[1]
# file_path='dataset/ADNI.mat'
file_path='dataset/PPMI.mat'
mat_data=scipy.io.loadmat(file_path)
print(file_path)
# print(mat_data.keys())
train_ratio=0.6
valid_ratio=0.2
test_ratio=0.2




Labels=[key for key in mat_data.keys() if not key.startswith('__')]
dict_labels = {label: i for i, label in enumerate(Labels)}
print(dict_labels)
scaler=StandardScaler()


import time
import random
# random_seed=int(time.time())

def data_process(random_seed):
    X_train=[]
    X_test=[]
    y_train=[]
    y_test=[]
    X_valid=[]
    y_valid=[]
    y_train_master=[] # 在主分类器中, 将 MCI, MCIn, MCIp 合并为一类的训练集标签
    y_valid_master=[]
    y_test_master=[]
    X_train_MCI=[] # MCI, MCIn, MCIp 的训练集, 验证集及其标签
    X_valid_MCI=[]
    y_train_MCI=[]
    y_valid_MCI=[]
    X_test_MCI=[]
    y_test_MCI=[]

    print(f'random seed: {random_seed}')
    np.random.seed(random_seed)

    for label,data in mat_data.items():
        if label.startswith('__'):
            continue
        label_num=dict_labels[label]
        N=np.shape(data)[0]
        indices=np.random.permutation(N)
        train_end = int(train_ratio * N)
        valid_end = int((train_ratio + valid_ratio) * N)

        train_index = indices[:train_end]
        validation_index = indices[train_end:valid_end]
        test_index = indices[valid_end:]
        # print(indices)
        # print(train_index,validation_index,test_index)
        for i in range(N):
            if i in train_index:
                X_train.append(data[i])
                y_train.append(label_num)
                if file_path=='dataset/ADNI.mat' and label_num>=1 and label_num<=3:
                    y_train_master.append(-1)
                    X_train_MCI.append(data[i]) # MCI, MCIn, MCIp 的训练集
                    y_train_MCI.append(label_num)
                else:
                    y_train_master.append(label_num)
            if i in validation_index:
                X_valid.append(data[i])
                y_valid.append(label_num)
                if file_path=='dataset/ADNI.mat' and label_num>=1 and label_num<=3:
                    y_valid_master.append(-1)
                    X_valid_MCI.append(data[i]) # MCI, MCIn, MCIp 的验证集
                    y_valid_MCI.append(label_num)
                else:
                    y_valid_master.append(label_num)
            if i in test_index:
                X_test.append(data[i])
                y_test.append(label_num)
                if file_path=='dataset/ADNI.mat' and label_num>=1 and label_num<=3:
                    y_test_master.append(-1)
                    X_test_MCI.append(data[i]) # MCI, MCIn, MCIp 的测试集
                    y_test_MCI.append(label_num)
                else:
                    y_test_master.append(label_num)
                

    dict_labels_MCI = {label: i for i, label in enumerate(Labels[1:])}
    X_train = np.array(X_train)
    y_train=np.array(y_train)
    y_train_master=np.array(y_train_master)

    X_valid = np.array(X_valid)
    y_valid=np.array(y_valid)
    y_valid_master=np.array(y_valid_master)

    X_train_MCI = np.array(X_train_MCI)
    y_train_MCI=np.array(y_train_MCI)

    X_valid_MCI = np.array(X_valid_MCI)
    y_valid_MCI=np.array(y_valid_MCI)

    X_test = np.array(X_test)
    y_test=np.array(y_test)
    y_test_master=np.array(y_test_master)
    def normalize(X):
        if X!=[]:
            scaler = StandardScaler()
            X = scaler.fit_transform(X)
            # X=(X-np.mean(X,axis=0))/np.std(X,axis=0)
        return X
    Labels_MCI=[-1,0,4]
    # X_train=normalize(X_train)
    # X_valid=normalize(X_valid)
    # X_train_MCI=normalize(X_train_MCI)
    # X_valid_MCI=normalize(X_valid_MCI)
    # X_test=normalize(X_test)
    # X_test_MCI=normalize(X_test_MCI)
    return X_train,y_train,X_valid,y_valid,X_test,y_test

dataset/PPMI.mat
{'NC': 0, 'PD': 1}


In [ ]:

import sys,os
# sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(__file__)))) # 添加上层目录到 sys.path
# from data_process import data_process,Labels
# from data_process import X_test,X_train
# from data_process import y_test,y_train,Labels
# from data_process import X_test_MCI,X_train_MCI,y_test_MCI,y_train_MCI
# # from dimension_reduction import X_train,X_test
from dimension_reduction_lda import dimension_reduction,visualization
import numpy as np
from sklearn import svm
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
dim=2

def _svm(X_train,y_train,X_valid,y_valid,X_test,y_test):
        clf=svm.SVC(kernel='linear',C=1.0,random_state=0,decision_function_shape='ovr')
        clf.fit(X_train,y_train)
        y_pred=clf.predict(X_test)
        train_acc=accuracy_score(y_train,clf.predict(X_train))
        valid_acc=accuracy_score(y_valid,clf.predict(X_valid))
        test_acc=accuracy_score(y_test,y_pred)
        # print('train:',accuracy_score(y_train,clf.predict(X_train)))
        # print('validation',accuracy_score(y_valid,clf.predict(X_valid)))
        # print('test:',accuracy_score(y_test,clf.predict(X_test)))
        # print(classification_report(y_test,y_pred,target_names=Labels))
        return train_acc,valid_acc,test_acc
seeds=[0,1,2,3,4]
train_accs,valid_accs,test_accs=[],[],[]
# if __name__=='__main__':
for seed in seeds:
    # X_train, y_train, X_valid, y_valid, X_test, y_test=[],[],[],[],[],[]
    X_train, y_train, X_valid, y_valid, X_test, y_test=data_process(seed)
    print('降维前的svm的表现:')
    _svm(X_train,y_train,X_valid, y_valid,X_test,y_test)
    print('降维后svm的表现:')

    X=np.concatenate((X_train,X_valid,X_test),axis=0)
    y=np.concatenate((y_train,y_valid,y_test),axis=0)
    X_reduced=dimension_reduction(X,y,dim=120,method='PCA')
    X_train_reduced=X_reduced[:X_train.shape[0],:]
    X_valid_reduced=X_reduced[X_train.shape[0]:X_train.shape[0]+X_valid.shape[0],:]
    X_test_reduced=X_reduced[X_train.shape[0]+X_valid.shape[0]:,:]

    # X_train_reduced=dimension_reduction(X_train_reduced,y_train,dim=30,method='Laplacian')
    # X_train_reduced,proj_mat=dimension_reduction(X_train_reduced,y_train,dim=1,method='LDA')
    # visualization(X_train_reduced,y_train,dim=dim,method='PCA',class_num=2)
    # X_test_reduced=dimension_reduction(X_test,y_test,dim=2,method='Isomap')
    # X_test_reduced=dimension_reduction(X_test_reduced,y_test,dim=30,method='Laplacian')
    # X_test_reduced=np.dot(X_test_reduced,proj_mat)
    # visualization(X_test_reduced,y_test,dim=dim,method='PCA',class_num=2)
    # _svm(X_train_reduced,y_train,X_test_reduced,y_test)
    train_acc,valid_acc,test_acc=_svm(X_train_reduced,y_train,X_valid_reduced,y_valid,X_test_reduced,y_test)
    train_accs.append(train_acc)
    valid_accs.append(valid_acc)
    test_accs.append(test_acc)
print('train_accs:',train_accs)
print('valid_accs:',valid_accs)
print('test_accs:',test_accs)
print('train_acc:',np.mean(train_accs),np.std(train_accs))
print('valid_acc:',np.mean(valid_accs),np.std(valid_accs))
print('test_acc:',np.mean(test_accs),np.std(test_accs))

dataset/PPMI.mat
{'NC': 0, 'PD': 1}
random seed: 0
降维前的svm的表现:
降维后svm的表现:
before dimension reduction: (543, 294)
